<a href="https://colab.research.google.com/github/kenowr/DCASE-2020-Task-1B/blob/master/Copy_of_MIT_6_036_HW02_part_1_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MIT 6.036 Fall 2021: Homework 02 Part 1#

**If you haven't already, please hit :**

`File` -> `Save a Copy in Drive`

**to copy this notebook to your Google drive, and work on a copy. If you don't do this, your changes won't be saved!**

---


This colab notebook provides code and a framework for problems 1-6 of the homework.  You can work out your solutions here, then submit your results back on the homework page when ready.


## <section>**Setup**</section>


Run the next code block to download and import the code for this lab.

In [2]:
!rm -f code_for_hw02.py*
!wget --no-check-certificate --quiet https://introml.odl.mit.edu/cat-soop/_static/6.036/homework/hw02/code_for_hw02.py 
from code_for_hw02 import *
import numpy as np

## 4) Implement Perceptron

Implement [the Perceptron algorithm](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2020_Fall/courseware/Week2/perceptron/2), where

* `data` is a numpy array of dimension $d$ by $n$
* `labels` is numpy array of dimension $1$ by $n$
* `params` is a dictionary specifying extra parameters to this algorithm; your algorithm should run a number of iterations equal to $T$


It should return a tuple of $\theta$ (a $d$ by 1 array) and $\theta_0$ (a 1 by 1 array).

We have given you some  data sets in the code file for you to test your implementation. Below are some test cases.
```
# Test Case 1
>>> data = np.array([[2, 3, 9, 12],
                     [5, 2, 6, 5]])
>>> labels = np.array([[1, -1, 1, -1]])
>>> [x.tolist() for x in perceptron(data, labels, {"T": 100})]
[[[-24.0], [37.0]], [[-3.0]]]

# Test Case 2
>>> data = np.array([[1, 2, 1, 2],
                     [1, 2, 2, 1]])
>>> labels = np.array([[1, 1, -1, -1]])
>>> [x.tolist() for x in perceptron(data, labels, {"T": 100})]
[[[0.0], [-3.0]], [[0.0]]]
```

Your function should initialize any parameters defined in the function to 0, then run through the data, in the order it is given, performing an update to the parameters whenever the current parameters would make a mistake on that data point. Perform `T` iterations through the data.

In [3]:
def perceptron(data, labels, params={}):
    # if T not in params, default to 100
    T = params.get('T', 100)

    # Your implementation here
    num_row, num_col = np.array(np.shape(data))
    th = np.zeros((num_row,1))
    th0 = np.zeros((1,1))

    for i1 in range(T):
      changed = False

      for i2 in range(num_col):
        if labels[0,i2] * (th.T @ data[:,i2:i2+1] + th0) <= 0.0:
          th = th + labels[0:,i2] * data[:,i2:i2+1]
          th0 = th0 + labels[0:,i2:i2+1]
          changed = True

      if changed == False:
        break

    return (th, th0)

In [4]:
test_perceptron(perceptron)

-----------Test Perceptron 0-----------
Passed! 

-----------Test Perceptron 1-----------
Passed! 



# 5) Implement Averaged Perceptron

We know that perceptron is guaranteed to find a separator if the data is linearly separable, but what can we do if the data is not linearly separable! Finding the separator that minimizes the number of mistakes could take an amount of computation that grows exponentially with the number of training examples, so we might settle for an approximate solution. Simply running the perceptron algorithm for a while, and then stopping and returning the result is not the best strategy, because the output hypothesis is highly influenced by the data points it has seen most recently. We can make a version of the perceptron that’s more stable and robust for non-separable data by returning the average value of th and th0 across all iterations. This is called the averaged perceptron algorithm.

Implement averaged Perceptron with the same spec as regular Perceptron (implemented above).

Below are a couple of example test cases.
```
# Test Case 1
>>> data = np.array([[2, 3, 9, 12],
                     [5, 2, 6, 5]])
>>> labels = np.array([[1, -1, 1, -1]])
>>> [x.tolist() for x in averaged_perceptron(data, labels, {"T": 100})]
[[[-22.1925], [34.06]], [[-2.1725]]]

# Test Case 2
>>> data = np.array([[1, 2, 1, 2],
                     [1, 2, 2, 1]])
>>> labels = np.array([[1, 1, -1, -1]])
>>> [x.tolist() for x in averaged_perceptron(data, labels, {"T": 100})]
[[[1.47], [-1.7275]], [[0.985]]]
```


Implement averaged perceptron with the same spec as regular perceptron.

In [5]:
def averaged_perceptron(data, labels, params={}):
    # if T not in params, default to 100
    T = params.get('T', 100)

    # Your implementation here
    num_row, num_col = np.array(np.shape(data))
    th = np.zeros((num_row,1))
    th0 = np.zeros((1,1))
    th_running = np.zeros((num_row,1))
    th0_running = np.zeros((1,1))
    counter = 0

    for i1 in range(T):
      changed = False

      for i2 in range(num_col):
        if labels[0,i2] * (th.T @ data[:,i2:i2+1] + th0) <= 0:
          th = th + labels[0:,i2] * data[:,i2:i2+1]
          th0 = th0 + labels[0:,i2:i2+1]
          changed = True
      
        th_running[:,0] = th_running[:,0] + th[:,0]
        th0_running[:,0] = th0_running[:,0] + th0
        counter = counter + 1 
      
      if changed == False:
        break

    return (th_running/counter, th0_running/counter)

In [6]:
test_averaged_perceptron(averaged_perceptron)

-----------Test Averaged Perceptron 0-----------
Test Failed.
Your code output  th: [[-9.875], [11.041666666666666]], th0: [[1.0416666666666667]]
Expected  th: [[-9.0525], [17.5825]], th0: [[1.9425]]


-----------Test Averaged Perceptron 1-----------
Passed! 



# 6) Implement Evaluation Strategies
  
## 6.1)  Evaluating a Classifier

To evaluate a classifier, we are interested in how well it performs on data that it wasn't trained on. Construct a testing procedure that uses a training data set, calls a learning algorithm to get a linear separator (a tuple of $\theta, \theta_0$), and then reports the percentage correct on a new testing set as a float between $0.0$ and $1.0$.

The learning algorithm is passed as a function that takes a data array and a labels vector.  Your evaluator should be able to interchangeably evaluate `perceptron` or `averaged_perceptron` (or future algorithms with the same spec), depending on what is passed through the `learner` parameter.

Assume that you have available the function `score` from HW 1, which takes inputs:

* `data`: a `d` by `n` array of floats (representing `n` data points in `d` dimensions)
* `labels`: a `1` by `n` array of elements in `(+1, -1)`, representing target labels
* `th`: a `d` by `1` array of floats that together with `th0`, represents a hyperplane
* `th0`: a single scalar or `1` by `1` array

and returns a scalar number of data points that the separator correctly classified.

The `eval_classifier` function should accept the following parameters:

* `learner` - a function, such as `perceptron` or `averaged_perceptron`
* `data_train` - training data
* `labels_train` - training labels
* `data_test` - test data
* `labels_test` - test labels

and returns the percentage correct on a new testing set as a float between $0.0$ and $1.0$.


In [7]:
def eval_classifier(learner, data_train, labels_train, data_test, labels_test):
    (th, th0) = learner(data_train, labels_train, params={})
    total_score = score(data_test, labels_test, th, th0)
    (num_rows, num_cols) = data_test.shape
    return total_score / num_cols

In [8]:
test_eval_classifier(eval_classifier, perceptron)

-----------Test Eval Classifier 0-----------
Passed! 

-----------Test Eval Classifier 1-----------
Passed! 



### 6.2) Evaluating a Learning algorithm using a data source

Construct a testing procedure that takes a learning algorithm and a data source as input and runs the learning algorithm multiple times, each time evaluating the resulting classifier as above. It should report the overall average classification accuracy.

You can use our implementation of `eval_classifier` by viewing the answer for that question and copying its definition into your code cell.

Write the function `eval_learning_alg` that takes:

* `learner` - a function, such as `perceptron` or `averaged_perceptron`
* `data_gen` - a data generator, call it with a desired data set size; returns a tuple `(data, labels)`
* `n_train` - the size of the learning sets
* `n_test` - the size of the test sets
* `it` - the number of iterations to average over

and returns the average classification accuracy as a float between $0.0$ and $1.0$.

**Note: Be sure to generate your training data separately before testing data, to ensure that the pseudo-randomly generated data matches that in the test code.**

In [9]:
def eval_classifier(learner, data_train, labels_train, data_test, labels_test):
    (th, th0) = learner(data_train, labels_train, params={})
    total_score = score(data_test, labels_test, th, th0)
    (num_rows, num_cols) = data_test.shape
    return total_score / num_cols

def eval_learning_alg(learner, data_gen, n_train, n_test, it):
    score = 0.0
    for i1 in range(it):
      (data_train, labels_train) = data_gen(n_train) 
      (data_test, labels_test) = data_gen(n_test)
      score = score + eval_classifier(learner, data_train, labels_train, data_test, labels_test)  
    return score/it

In [10]:
test_eval_learning_alg(eval_learning_alg, perceptron)

-----------Test Eval Learning Algo-----------
Passed! 



## 6.3) Evaluating a Learning Algorithm With a Fixed Dataset

Cross-validation is a strategy for evaluating a learning algorithm, using a single training set of size $n$. Cross-validation takes in a learning algorithm $L$, a fixed data set $\mathcal{D}$, and a parameter $k$. It will run the learning algorithm $k$ different times, then evaluate the accuracy of the resulting classifier, and ultimately return the average of the accuracies over each of the $k$ "runs" of $L$. It is structured like this:

<pre><code>divide D into k parts, as equally as possible;  call them D_i for i == 0 .. k-1
# be sure the data is shuffled in case someone put all the positive examples first in the data!
for j from 0 to k-1:
    D_minus_j = union of all the datasets D_i, except for D_j
    h_j = L(D_minus_j)
    score_j = accuracy of h_j measured on D_j
return average(score0, ..., score(k-1))</code></pre>

So, each time, it trains on  $k−1$ of the pieces of the data set and tests the resulting hypothesis on the piece that was not used for training.

When $k=n$, it is called *leave-one-out cross validation*.

Implement cross validation **assuming that the input data is shuffled already** so that the positives and negatives are distributed randomly. If the size of the data does not evenly divide by k, split the data into `n % k` sub-arrays of size `n // k + 1` and the rest of size `n // k`.

You can use <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.array_split.html">np.array_split</a>
and <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.concatenate.html">np.concatenate</a> with axis arguments to split and rejoin the data as you desire. You can also use our implementation of `eval_classifier` by viewing the answer for that question and copying its definition into your code cell.

Note: In Python, `n//k` indicates integer division, e.g. `2//3 = 0` and `4//3 = 1`.

In [11]:
def xval_learning_alg(learner, data, labels, k):
    # Cross validation of learning algorithm
    # Your implementation here
    (d, n) = data.shape
    n_section = n // k    # Size of each section
    data_new = np.array_split(data, k, axis=1)
    labels_new = np.array_split(labels, k, axis=1)
    score = np.zeros((k,1))
   
    for i1 in range(k):
      data_test = data_new[i1]      # Test data
      labels_test = labels_new[i1]  # Test data
      init = True
      
      for i2 in range(k):
        if i2 != i1:                # Exclusion of test data
          if init == True:
            data_train = data_new[i2]   # Initialise
            labels_train = labels_new[i2]
            init = False
          else:  
            data_train = np.concatenate((data_train, data_new[i2]), axis=1)
            labels_train = np.concatenate((labels_train, labels_new[i2]), axis=1)

      score[i1] = eval_classifier(learner, data_train, labels_train, data_test, labels_test)
    
    return np.mean(score)
    
    


   
  


In [12]:
test_xval_learning_alg(xval_learning_alg, perceptron)

-----------Test Cross-eval Learning Algo-----------
Passed! 

